To start using an agent we need to create an LLM that will perform Natural Language processing

In [36]:
from langchain.llms import AzureOpenAI
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

# we are using not a chat based llm, but a simple Davinci
llm = AzureOpenAI(
    openai_api_version="2023-03-15-preview",
    deployment_name="Davinci003", 
    model_name="text-davinci-003"
)


Now lets try to use the calculator.

In [ ]:
from langchain.chains import LLMMathChain
from langchain.agents import Tool
from pprint import pprint

llm_math = LLMMathChain(llm=llm)

# initialize the math tool
math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful for when you need to answer questions about math.'
)
# when giving tools to LLM, we must pass as list of tools
tools = [math_tool]

# prints information about the tools using tools[0].name, tools[0].description
pprint(f"Name: {tools[0].name}\nDescription: {tools[0].description}")


Usually you can simply use a predefined string identifier for specific tool like calculator

In [ ]:
from langchain.agents import load_tools

tools = load_tools(
    ['llm-math'],
    llm=llm
)

pprint(f"Name: {tools[0].name}\nDescription: {tools[0].description}")

In [ ]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)


In [ ]:
zero_shot_agent("what is (4.5*2.1)^2.2?")

In [ ]:
zero_shot_agent("if Mary has four apples and Giorgio brings two and a half apple "
                "boxes (apple box contains eight apples), how many apples do we "
                "have?")

If you ask questions that has no tools, the chain cannot answer

In [ ]:
zero_shot_agent("what is the capital of Norway?")

The code BELOW is a Python script that imports two modules, `PromptTemplate` and `LLMChain`, from the `langchain.prompts` and `langchain.chains` packages, respectively. The script then defines a `PromptTemplate` object with an input variable named `query` and a template string that simply returns the value of `query`. 

Next, the script creates an instance of the `LLMChain` class, passing in an LLM model and the `PromptTemplate` object as arguments. The `LLMChain` class is a wrapper around the LLM model that provides a simple interface for generating text based on a given prompt.

Finally, the script creates a `Tool` object, which is a custom class that represents a tool in a larger system. The `Tool` object is initialized with a name, a function to run when the tool is called, and a description. In this case, the function is the `run` method of the `LLMChain` instance created earlier, which generates text based on the input prompt.

Overall, this code is an example of how to use the `langchain` package to create a custom tool that generates text based on a given prompt. The `PromptTemplate` and `LLMChain` classes provide a simple and flexible way to generate text using an LLM model, while the `Tool` class provides a way to integrate this functionality into a larger system.

To improve the readability of this code, it might be helpful to add comments explaining the purpose of each section of code. Additionally, it might be useful to break up the code into smaller functions or classes to make it easier to understand and modify.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["query"],
    template="{query}"
)

llm_chain = LLMChain(llm=llm, prompt=prompt)

# initialize the LLM tool, this is a really generic tool than can query logic using another the very same llm.
llm_tool = Tool(
    name='Language Model',
    func=llm_chain.run,
    description='use this tool for general purpose queries and logic'
)


In [ ]:
tools = load_tools(
    ['llm-math'],
    llm=llm
)

tools.append(llm_tool)

# reinitialize the agent
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)


Now we have a generic tool that can simply answer to a question directly

In [ ]:
zero_shot_agent("what is the capital of Norway?")

In [ ]:
from sqlalchemy import MetaData

metadata_obj = MetaData()
from sqlalchemy import Column, Integer, String, Table, Date, Float

stocks = Table(
    "stocks",
    metadata_obj,
    Column("obs_id", Integer, primary_key=True),
    Column("stock_ticker", String(4), nullable=False),
    Column("price", Float, nullable=False),
    Column("date", Date, nullable=False),    
)
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)
from datetime import datetime

observations = [
    [1, 'ABC', 200, datetime(2023, 1, 1)],
    [2, 'ABC', 208, datetime(2023, 1, 2)],
    [3, 'ABC', 232, datetime(2023, 1, 3)],
    [4, 'ABC', 225, datetime(2023, 1, 4)],
    [5, 'ABC', 226, datetime(2023, 1, 5)],
    [6, 'XYZ', 810, datetime(2023, 1, 1)],
    [7, 'XYZ', 803, datetime(2023, 1, 2)],
    [8, 'XYZ', 798, datetime(2023, 1, 3)],
    [9, 'XYZ', 795, datetime(2023, 1, 4)],
    [10, 'XYZ', 791, datetime(2023, 1, 5)],
]
from sqlalchemy import insert

def insert_obs(obs):
    stmt = insert(stocks).values(
    obs_id=obs[0], 
    stock_ticker=obs[1], 
    price=obs[2],
    date=obs[3]
    )

    with engine.begin() as conn:
        conn.execute(stmt)
for obs in observations:
    insert_obs(obs)

In [ ]:
from langchain.agents import initialize_agent

tools = load_tools(
    ["llm-math"], 
    llm=llm
)

# add our custom SQL db tool
tools.append(sql_tool)

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description", 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=3,
)


Try with memory

In [28]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

conversational_agent = initialize_agent(
    agent='conversational-react-description', 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,
)



Now try to query wikipedia

In [30]:
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer

docstore=DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description='search wikipedia'
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description='lookup a term in wikipedia'
    )
]

docstore_agent = initialize_agent(
    tools, 
    llm, 
    agent="react-docstore", 
    verbose=True,
    max_iterations=6
)


In [32]:
docstore_agent("What were Archimedes' last words?")



> Entering new  chain...
Thought: I need to search Archimedes and find out his last words.
Action: Search[Archimedes]
Observation: Archimedes of Syracuse (; c. 287 – c. 212 BC) was an Ancient Greek mathematician, physicist, engineer, astronomer, and inventor from the ancient city of Syracuse in Sicily. Although few details of his life are known, he is regarded as one of the leading scientists in classical antiquity. Considered the greatest mathematician of ancient history, and one of the greatest of all time, Archimedes anticipated modern calculus and analysis by applying the concept of the infinitely small and the method of exhaustion to derive and rigorously prove a range of geometrical theorems. These include the area of a circle, the surface area and volume of a sphere, the area of an ellipse, the area under a parabola, the volume of a segment of a paraboloid of revolution, the volume of a segment of a hyperboloid of revolution, and the area of a spiral.Archimedes' other mathemat

{'input': "What were Archimedes' last words?",
 'output': '"Do not disturb my circles"'}

Now try to perform a search on google

In [38]:
from langchain import SerpAPIWrapper
import os
# initialize the search chain
search = SerpAPIWrapper(serpapi_api_key=os.getenv("SERPAPI_API_KEY"))

# create a search tool
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description='google search'
    )
]

# initialize the search enabled agent
self_ask_with_search = initialize_agent(
    tools,
    llm,
    agent="self-ask-with-search",
    verbose=True
)


In [39]:
self_ask_with_search(
    "who lived longer; Plato, Socrates, or Aristotle?"
)



> Entering new  chain...
 Yes.
Follow up: How old was Plato when he died?
Intermediate answer: eighty
Follow up: How old was Socrates when he died?
Intermediate answer: At the age of 70, he was put to death at the hands of his fellow citizens on charges of impiety and corruption of the youth. His trial, along with the social and political context in which occurred, has warranted as much treatment from historians and classicists as his arguments and methods have from philosophers.
Follow up: How old was Aristotle when he died?
Intermediate answer: 62 years
So the final answer is: Plato

> Finished chain.


{'input': 'who lived longer; Plato, Socrates, or Aristotle?',
 'output': 'Plato'}